# Example dashboard 
Using the code and data available at https://github.com/voila-dashboards/voila-gridstack/blob/main/examples/scotch_dashboard.ipynb

In [ ]:
## NOTE: TO RUN USE THE FOLLOWING LINE IN COMMAND WINDOW -> need to ensure gridstack works
# voila .\dashboard.ipynb  --VoilaConfiguration.resources="{'gridstack': {'show_handles': True}}"

In [13]:
#Need to ensure:
#   - installed packages/software with pip install
#    - python
#    - pandas
#    - voila (might need to check version of pygments as error might occur with pygments <2.4.1, while voila requires pygments >=2.4.1.)
#    - voila-gridstack
#    - ipywidgets
#    - bqplot

In [4]:
import ipywidgets as widgets
from traitlets import Unicode, List, Instance, link, HasTraits
from IPython.display import display, clear_output, HTML, Javascript

In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
# Now import data

In [7]:
# import data
curr_dir= os.getcwd()
data_path=os.path.join(curr_dir,'data')

scotch_filename=os.path.join(data_path, 'scotch_data.csv')
scotch_df=pd.read_csv(scotch_filename, index_col=0)

In [13]:
import sys
basic_science_path=os.path.join(curr_dir,'code_of_example_package','example_package','example_package')
sys.path.insert(0, basic_science_path)


In [ ]:
import basic_science

In [12]:
scotch_df_normed=basic_science.get_normed_data(scotch_df)

In [ ]:
# Now run and set up parts of gridstack dashboard

In [ ]:
def radar(df, ax=None):
    # calculate evenly-spaced axis angles
    num_vars = len(df.columns)
    theta = 2*np.pi * np.linspace(0, 1-1./num_vars, num_vars)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2
    if not ax:
        fig = plt.figure(figsize=(4, 4))

        ax = fig.add_subplot(1,1,1, projection='polar')
    else:
        ax.clear()
    for d, color in zip(df.itertuples(), sns.color_palette()):
        ax.plot(theta, d[1:], color=color, alpha=0.7)
        ax.fill(theta, d[1:], facecolor=color, alpha=0.5)
    ax.set_xticklabels(df.columns)

    legend = ax.legend(df.index, loc=(0.9, .95))
    return ax

In [ ]:
def get_similar(name, n, top=True):
    a = scotch_df_normed[name].sort_values(ascending=False)
    a.name = 'Similarity'
    df = pd.DataFrame(a) #.join(scotch_df).iloc[start:end]
    return df.head(n) if top else df.tail(n)

In [ ]:
def on_pick_scotch(Scotch):
    name = Scotch
    # Get top 6 similar whiskeys, and remove this one
    top_df = get_similar(name, 6).iloc[1:]
    # Get bottom 5 similar whiskeys
    df = top_df
    
    # Make table index a set of links that the radar widget will watch
    df.index = ['''<a class="scotch" href="#" data-factors_keys='["{}","{}"]'>{}</a>'''.format(name, i, i) for i in df.index]
    
    tmpl = f'''<p>If you like {name} you might want to try these five brands. Click one to see how its taste profile compares.</p>'''
    prompt_w.value = tmpl
    table.value = df.to_html(escape=False)
    lines.x = scotch_df.loc[Scotch].index.values
    lines.y = scotch_df.loc[Scotch].values

In [ ]:
prompt_w = widgets.HTML(value='Aberfeldy')
display(prompt_w)

In [ ]:
table = widgets.HTML(
    value="Hello <b>World</b>"
)
display(table)

In [ ]:
from bqplot import (OrdinalScale, LinearScale, Bars, Lines,
                    Figure, Axis, ColorScale, ColorAxis, CATEGORY10)
x_ord = OrdinalScale()
y_sc = LinearScale()

lines = Lines(x=scotch_df.loc['Aberfeldy'].index.values,
              y=scotch_df.loc['Aberfeldy'].values, scales={'x': x_ord, 'y': y_sc},
              fill='bottom', fill_colors=['#aaaaff'], fill_opacities=[0.4],
              stroke_width=3)
ax_x = Axis(scale=x_ord, tick_rotate=45, tick_style={'font-size': 20})
ax_y = Axis(scale=y_sc, tick_format='0.2f', orientation='vertical')

Figure(marks=[lines], axes=[ax_x, ax_y], animation_duration=500)

In [ ]:
picker_w = widgets.interact(on_pick_scotch,  Scotch=list(scotch_df_normed.index))